In [6]:
import argparse
import sys

import os
import random
import scipy as sp
import pickle

import shutil
import csv
import ast
import community as community_louvain


import scipy.sparse as sparse
from tqdm import tqdm
from torch import Tensor
import networkx as nx
import numpy as np
from datetime import datetime
import torch
import torch.nn as nn
from torch_geometric.data import Data

import torch.nn.functional as F
from torch_geometric.loader import DataLoader

from autoencoder import VariationalAutoEncoder
from denoise_model import DenoiseNN, p_losses, sample
from utils import linear_beta_schedule, construct_nx_from_adj, preprocess_dataset, graph_statistics
from sklearn.metrics import mean_absolute_error
from scipy.stats import zscore




from torch.utils.data import Subset
np.random.seed(13)


In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
# Simulate command-line arguments
sys.argv = [
    'main.py',
    '--lr', '0.001',
    '--dropout', '0.1',
    '--batch-size', '256',
    '--epochs-autoencoder', '200',
    '--hidden-dim-encoder', '64',
    '--hidden-dim-decoder', '256',
    '--latent-dim', '32',
    '--n-max-nodes', '50',
    '--n-layers-encoder', '2',
    '--n-layers-decoder', '3',
    '--spectral-emb-dim', '10',
    '--epochs-denoise', '100',
    '--timesteps', '500',
    '--hidden-dim-denoise', '512',
    '--n-layers_denoise', '3',
    # '--train-autoencoder',
    '--train-denoiser',
    '--dim-condition', '128',
    '--n-condition', '7'
]

# Initialize the parser
parser = argparse.ArgumentParser(description="Your description here")

# Add arguments
parser.add_argument('--lr', type=float, default=1e-3, help="Learning rate for the optimizer, typically a small float value (default: 0.001)")
parser.add_argument('--dropout', type=float, default=0.0, help="Dropout rate (fraction of nodes to drop) to prevent overfitting (default: 0.0)")
parser.add_argument('--batch-size', type=int, default=256, help="Batch size for training, controlling the number of samples per gradient update (default: 256)")
parser.add_argument('--epochs-autoencoder', type=int, default=200, help="Number of training epochs for the autoencoder (default: 200)")
parser.add_argument('--hidden-dim-encoder', type=int, default=64, help="Hidden dimension size for encoder layers (default: 64)")
parser.add_argument('--hidden-dim-decoder', type=int, default=256, help="Hidden dimension size for decoder layers (default: 256)")
parser.add_argument('--latent-dim', type=int, default=32, help="Dimensionality of the latent space in the autoencoder (default: 32)")
parser.add_argument('--n-max-nodes', type=int, default=50, help="Possible maximum number of nodes in graphs (default: 50)")
parser.add_argument('--n-layers-encoder', type=int, default=2, help="Number of layers in the encoder network (default: 2)")
parser.add_argument('--n-layers-decoder', type=int, default=3, help="Number of layers in the decoder network (default: 3)")
parser.add_argument('--spectral-emb-dim', type=int, default=10, help="Dimensionality of spectral embeddings for representing graph structures (default: 10)")
parser.add_argument('--epochs-denoise', type=int, default=100, help="Number of training epochs for the denoising model (default: 100)")
parser.add_argument('--timesteps', type=int, default=500, help="Number of timesteps for the diffusion (default: 500)")
parser.add_argument('--hidden-dim-denoise', type=int, default=512, help="Hidden dimension size for denoising model layers (default: 512)")
parser.add_argument('--n-layers_denoise', type=int, default=3, help="Number of layers in the denoising model (default: 3)")
parser.add_argument('--train-autoencoder', action='store_true', default=False, help="Flag to enable/disable autoencoder (VGAE) training (default: enabled)")
parser.add_argument('--train-denoiser', action='store_true', default=False, help="Flag to enable/disable denoiser training (default: enabled)")
parser.add_argument('--dim-condition', type=int, default=128, help="Dimensionality of conditioning vectors for conditional generation (default: 128)")
parser.add_argument('--n-condition', type=int, default=7, help="Number of distinct condition properties used in conditional vector (default: 7)")

# Parse the arguments
args = parser.parse_args()

# Use the arguments as needed
print(args)

Namespace(lr=0.001, dropout=0.1, batch_size=256, epochs_autoencoder=200, hidden_dim_encoder=64, hidden_dim_decoder=256, latent_dim=32, n_max_nodes=50, n_layers_encoder=2, n_layers_decoder=3, spectral_emb_dim=10, epochs_denoise=100, timesteps=500, hidden_dim_denoise=512, n_layers_denoise=3, train_autoencoder=False, train_denoiser=True, dim_condition=128, n_condition=7)


In [8]:
from transformers import AutoTokenizer, ModernBertModel

model_id = "answerdotai/ModernBERT-base"
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = ModernBertModel.from_pretrained(model_id)

model.to("cuda")

text = "The capital of France is [MASK]."
inputs = tokenizer(text, return_tensors="pt")
inputs.to("cuda")


Flash Attention 2.0 only supports torch.float16 and torch.bfloat16 dtypes, but the current dype in ModernBertModel is torch.float32. You should run training or inference using Automatic Mixed-Precision via the `with torch.autocast(device_type='torch_device'):` decorator, or load the model with the `torch_dtype` argument. Example: `model = AutoModel.from_pretrained("openai/whisper-tiny", attn_implementation="flash_attention_2", torch_dtype=torch.float16)`


{'input_ids': tensor([[50281,   510,  5347,   273,  6181,   310, 50284,    15, 50282]],
       device='cuda:0'), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1]], device='cuda:0')}

In [19]:

outputs = model(**inputs)
print(outputs.last_hidden_state[:, 0, :].shape) #CLS token


torch.Size([1, 768])


In [7]:
trainset = preprocess_dataset("train", args.n_max_nodes, args.spectral_emb_dim)
validset = preprocess_dataset("valid", args.n_max_nodes, args.spectral_emb_dim)
testset = preprocess_dataset("test", args.n_max_nodes, args.spectral_emb_dim)


/users/eleves-b/2021/valentin.dorseuil/Desktop/altegrad_project/new_loss/utils.py:80: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  print(f'Dataset {filename} loaded from fi

Dataset ./data/dataset_train.pt loaded from file
Dataset ./data/dataset_valid.pt loaded from file
Dataset ./data/dataset_test.pt loaded from file


/users/eleves-b/2021/valentin.dorseuil/Desktop/altegrad_project/new_loss/utils.py:53: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  data_lst = torch.load(filename)


In [5]:
testset.__getitem__(0)

Data(stats=[1, 7], filename='graph_0')